<a href="https://colab.research.google.com/github/t0r8ern1t/comp_mod/blob/main/mm_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
def interpolate_row(row, method='linear'):
    if method == 'linear':
        return linear_interpolation(row)
    elif method == 'quadratic':
        return quadratic_interpolation(row)

def linear_interpolation(row):
    row = np.array(row, dtype=float)
    nans = np.isnan(row)
    for i in range(len(row)):
        if nans[i]:
            left = i - 1
            while left >= 0 and nans[left]:
                left -= 1
            right = i + 1
            while right < len(row) and nans[right]:
                right += 1

            if left >= 0 and right < len(row):
                row[i] = row[left] + (row[right] - row[left]) * (i - left) / (right - left)
            elif left >= 0:
                row[i] = row[left]
            elif right < len(row):
                row[i] = row[right]
    return row

def quadratic_interpolation(row):
    row = np.array(row, dtype=float)
    nans = np.isnan(row)
    for i in range(len(row)):
        if nans[i]:
            left1 = i - 1
            while left1 >= 0 and nans[left1]:
                left1 -= 1
            left2 = left1 - 1
            while left2 >= 0 and nans[left2]:
                left2 -= 1
            right1 = i + 1
            while right1 < len(row) and nans[right1]:
                right1 += 1
            right2 = right1 + 1
            while right2 < len(row) and nans[right2]:
                right2 += 1

            if left2 >= 0 and right2 < len(row):
                x = [left2, left1, right1, right2]
                y = [row[left2], row[left1], row[right1], row[right2]]
                denom = (x[0] - x[1]) * (x[0] - x[2]) * (x[1] - x[2])
                a = (y[2] * (x[1] - x[0]) + y[1] * (x[0] - x[2]) + y[0] * (x[2] - x[1])) / denom
                b = (y[2] * (x[0]**2 - x[1]**2) + y[1] * (x[2]**2 - x[0]**2) + y[0] * (x[1]**2 - x[2]**2)) / denom
                c = (y[2] * (x[1] * x[0]**2 - x[0] * x[1]**2) + y[1] * (x[2] * x[0]**2 - x[0] * x[2]**2) + y[0] * (x[1] * x[2]**2 - x[2] * x[1]**2)) / denom
                row[i] = a * i**2 + b * i + c
            elif left1 >= 0 and right1 < len(row):
                row[i] = row[left1] + (row[right1] - row[left1]) * (i - left1) / (right1 - left1)
            elif left1 >= 0:
                row[i] = row[left1]
            elif right1 < len(row):
                row[i] = row[right1]
    return row

def read_data(file_path):
    sheets = [pd.read_excel(file_path, sheet_name=i, header=None) for i in range(3)]
    return [sheet.iloc[2:34, 1:25].values.tolist() for sheet in sheets]

def write_data(file_path, data, method='linear'):
    with pd.ExcelWriter(file_path) as writer:
        for idx, dataset in enumerate(data):
            pd.DataFrame(dataset).to_excel(writer, sheet_name=f'Sheet{idx + 1}', header=False, index=False)

In [4]:
path_to_table = r"Data_2.xls"
data_sets = read_data(path_to_table)


linear_data = [[interpolate_row(row, 'linear') for row in data_set] for data_set in data_sets]
quadratic_data = [[interpolate_row(row, 'quadratic') for row in data_set] for data_set in data_sets]

# Запись результатов в файл
output_file = r"output.xlsx"
write_data(output_file, linear_data)
write_data(output_file, quadratic_data)

print(f"Результаты сохранены в файл: {output_file}")

Результаты сохранены в файл: output.xlsx
